In [3]:
from dotenv import load_dotenv
import os

from pymongo import MongoClient
load_dotenv()

clt = MongoClient(os.getenv('MONGO_URL'))

try:
    # start example code here
    # end example code here
    clt.admin.command("ping")
    print("Connected successfully")
    # other application code
    clt.close()
except Exception as e:
    raise Exception(
        "The following error occurred: ", e)

Connected successfully
